## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import scipy
import re
import nltk
from nltk.corpus import stopwords




## Reading the DataSet

In [2]:
df = pd.read_excel("Product Matching Dataset.xlsx" , sheet_name="Dataset")  
df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [3]:
test_df = pd.read_csv('test.csv')
df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [4]:
df.shape

(83562, 4)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83562 entries, 0 to 83561
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sku                          83562 non-null  int64  
 1   marketplace_product_name_ar  83562 non-null  object 
 2   seller_item_name             83562 non-null  object 
 3   price                        83562 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.6+ MB


In [6]:
df.describe()

,sku,price
count,83562.000000,83562.000000
mean,1600.653204,79.055458
std,1469.206223,62.818117
min,4.000000,7.000000
25%,476.000000,38.000000
50%,1312.000000,61.500000
75%,2258.000000,100.500000
max,9532.000000,406.000000


In [7]:
df.isna().sum()

sku                            0
marketplace_product_name_ar    0
seller_item_name               0
price                          0
dtype: int64

In [8]:
df = df.sample(frac=1).reset_index(drop=True)

In [9]:
df.columns

Index(['sku', 'marketplace_product_name_ar', 'seller_item_name', 'price'], dtype='object')

In [10]:
train_df, validation_df = train_test_split(df, test_size=0.2, stratify=df['sku'], random_state=42)

train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)

print("Train DataFrame shape:", train_df.shape)
print("Test DataFrame shape:", validation_df.shape)
print("\nTrain DataFrame first rows:\n", train_df.head())


Train DataFrame shape: (66849, 4)
Test DataFrame shape: (16713, 4)

Train DataFrame first rows:
     sku marketplace_product_name_ar             seller_item_name  price
0  1312      موزابرايد 5 مجم 30 قرص  موزابرايد 5 مجم 30 قرص جديد   91.5
1  4743     ميكروسيرك 16 مجم 20 قرص          ميكروسيرك 16 مجم 20   29.0
2  2517  البافيت كالسيوم شراب 60 مل   البافيت شراب كالسيوم/ابيكس   26.0
3  2374     زيستريل 20 مجم 10 اقراص   زيستريل 20مجم اقراص سعر 34   52.0
4   639           اتور 10 مجم 7 قرص   اتور 10مجم 7ق/ابيكو س جديد   33.0


In [11]:
# test_df = train_df[['seller_item_name' , 'price']]

In [12]:
# test_df.to_csv('test.csv' , index=False, encoding='utf-8-sig')

## Import train and test datasets

In [13]:
def arabic_word_preprocessing(text):
    # Download Arabic stopwords (only needed once)
    nltk.download('stopwords')

# Load Arabic stop words
    arabic_stopwords = set(stopwords.words('arabic'))
    X = []
    for i in range(len(text)):
        
        statment = re.sub(r'[إأآ]', 'ا', text[i])  
        statment = re.sub(r'ى', 'ي', statment)  
        statment = re.sub(r'ة', 'ه', statment)  
        statment = re.sub(r'ؤ', 'و', statment)  
        statment = re.sub(r'ئ', 'ي', statment)  
        statment = re.sub(r'ــ', '', statment)
        statment = re.sub(r'([\u0600-\u06FF])\1', r'\1', statment)
        
        pattern1 = '[^\u0621-\u064A0-9a-zA-Z\s]' 
        statment = re.sub(pattern1, ' ', statment)
        
        pattern = r"(.)\1+"
        statment = re.sub(pattern, r"\1", statment)
        
        
        statment = re.sub(r'(\d+)(?=\D)', r'\1 ', statment)
        statment = re.sub(r'(\D)(\d+)', r'\1 \2', statment)
        
        
        pattern2 = r'\b(اقراص\w*|قرص\w*|شري\w*|كبسول\w*|شرايط|افلام|فيلم|استحلاب|ك|ق)\b'
    
        statment =  re.sub(pattern2, 'قرص', statment)
        
        pattern10 = r'\bقرص\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern10, 'قرص ', statment)        
        
        
      
        
        patternn = r'\bامبول\w*|حقن\w*\b'
        
        statment = re.sub(patternn, 'امبول', statment)
        
        pattern11 = r'\bامبول\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern11, 'امبول', statment)
        
        pattern = r'جل'
        statment = re.sub(pattern, 'جيل', statment)
        
        pattern = r'جيل.*غسول'
        statment = re.sub(pattern, 'غسول', statment)
        
        pattern = r'دش'
        statment = re.sub(pattern, 'غسول', statment)
        
        pattern100 = r'\bجيل\s+(.+)\b'
        statment = re.sub(pattern100, 'جيل', statment)
        
        pattern100 = r'\bغسول\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'غسول', statment)
        
        pattern100 = r'\bمحلول\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'محلول', statment)
        

        # pattern20 = r'\bفروت|فواكه\b'
    
        # statment =  re.sub(pattern20, 'فواكه', statment)
        
        # pattern100 = r'\bفواكه\s+(.+)\b'
        # statment = re.sub(pattern100, 'فواكه', statment)
        
        pattern3 = r'\b(مل|ملي|مم|جم|مج|مجم|عادي|عاده)\b'
    
        statment =  re.sub(pattern3, '', statment)
        
        pattern = r'(\D)(قطر|قطره)\b'
        
        
        statment = re.sub(pattern, r'\1 نقط', statment)
        
        pattern = r'قطر\D+'
        statment = re.sub(pattern, r'نقط', statment)
        
        pattern30 = r'\b(قطره|قطر|نقط|نقطه|قطرهعين)\b'
    
        statment =  re.sub(pattern30, 'نقط', statment)
        

        
        pattern100 = r'\bنقط\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'نقط', statment)
        
        pattern = r'\sفوار|اكياس|اكيااس'
        statment = re.sub(pattern, 'كيس', statment)
        
        pattern100 = r'\bكيس\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'كيس', statment)
        
        pattern30 = r'\b(سبراي|بخاخه)\b'
    
        statment =  re.sub(pattern30, 'بخاخ', statment)
        
        
        pattern = r'مرهم|اكريم'
        statment = re.sub(pattern, 'كريم', statment)
        
        pattern100 = r'\bكريم\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'كريم', statment)
        
        
        # pattern = r'لوشن'
        # statment = re.sub(pattern, 'لوسيون', statment)
        
        # pattern100 = r'\bلوسيون\s+([^0-9\s]+)\s*'
        # statment = re.sub(pattern100, 'لوسيون', statment)
        
        pattern100 = r'\bبخاخ\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'بخاخ', statment)
        
        pattern99 = r'\b\w*استنشاق\w*\b'
        statment = re.sub(pattern99, '', statment)
        
        pattern100 = r'استنشـاق'
        statment = re.sub(pattern100, '', statment)
        
        patter = r'شـراب+|شرب+|شراب+'
        statment = re.sub(patter, ' شراب', statment)
        
        pattern100 = r'\bشراب\s+([^0-9\s]+)\s*'
        statment = re.sub(pattern100, 'شراب', statment)
        
       
        
        pattern4 = r'\b[قديم]+\b' 
        statment = re.sub(pattern4, '', statment)

        
        
        pattern5 = r'\b[جديد]+\b'
        statment = re.sub(pattern5,"", statment)
        
        pattern6 = r'\b[ء-ي]\b'
        statment = re.sub(pattern6, '', statment)
        
        pattern7 = r'\b[سعر]+\b'
        statment = re.sub(pattern7, '', statment)
        
        pattern8 = r'\bسج|سق\b'
        statment = re.sub(pattern8, '', statment)
        
        pattern9 = r'\b\w*سعر\w*\b'
        statment = re.sub(pattern9, '', statment)
        
        pattern10 = r'\b\w*جدي\w*\b'
        statment = re.sub(pattern10, '', statment)
        
        
        # statment = re.sub('بلاس', 'بلس', statment)
        
        pattern = r"(.)\1+"
        statment = re.sub(pattern, r"\1", statment)
        
        words = statment.split()

    
        filtered_words = [word for word in words if word not in arabic_stopwords]

    



        X.append(' '.join(filtered_words))
   
    
    return X

<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
/var/folders/qc/_v6f30pj3sq1vn0jhkyg29zw0000gn/T/ipykernel_78593/943379365.py:18: SyntaxWarning: invalid escape sequence '\s'
  pattern1 = '[^\u0621-\u064A0-9a-zA-Z\s]'


In [14]:
cv = CountVectorizer()
le = LabelEncoder()

In [15]:
def preparing_train_df(train_df):
    seller_name = train_df['seller_item_name'].apply(str)
    X_train_text = arabic_word_preprocessing(seller_name)
    X_train_text = cv.fit_transform(X_train_text).toarray()

    X_train_price = train_df[['price']].values  
    X_train_price = StandardScaler().fit_transform(X_train_price)  

    X_train_combined = np.hstack([X_train_text, X_train_price])  

    y_train = train_df['sku'].values
    y_train = le.fit_transform(y_train)

    return X_train_combined, y_train


In [16]:
def preparing_validation_df(df):
    seller_name = df['seller_item_name'].apply(str)
    X_test_text = arabic_word_preprocessing(seller_name)
    # modefied = pd.DataFrame(X_test_text) 
    X_test_text = cv.transform(X_test_text).toarray()  

    X_test_price = df[['price']].values  
    X_test_price = StandardScaler().fit_transform(X_test_price)  

    X_test_combined = np.hstack([X_test_text, X_test_price])  

    y_test = df['sku'].values
    y_test = le.transform(y_test)

    return X_test_combined, y_test


In [17]:
def preparing_test_df(df):
    seller_name = df['seller_item_name'].apply(str)
    X_test_text = arabic_word_preprocessing(seller_name)
    X_test_text = cv.transform(X_test_text).toarray()  

    X_test_price = df[['price']].values  
    X_test_price = StandardScaler().fit_transform(X_test_price)  

    X_test_combined = np.hstack([X_test_text, X_test_price])  


    return X_test_combined


In [18]:
print(train_df.columns)


Index(['sku', 'marketplace_product_name_ar', 'seller_item_name', 'price'], dtype='object')


In [19]:
print(train_df['seller_item_name'].isnull().sum()) 


0


In [20]:
print(train_df.shape)  


(66849, 4)


In [21]:
train_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1312,موزابرايد 5 مجم 30 قرص,موزابرايد 5 مجم 30 قرص جديد,91.5
1,4743,ميكروسيرك 16 مجم 20 قرص,ميكروسيرك 16 مجم 20,29.0
2,2517,البافيت كالسيوم شراب 60 مل,البافيت شراب كالسيوم/ابيكس,26.0
3,2374,زيستريل 20 مجم 10 اقراص,زيستريل 20مجم اقراص سعر 34,52.0
4,639,اتور 10 مجم 7 قرص,اتور 10مجم 7ق/ابيكو س جديد,33.0


In [22]:
type(df)

pandas.core.frame.DataFrame

In [23]:
type(train_df)

pandas.core.frame.DataFrame

In [24]:
df.shape

(83562, 4)

In [25]:
train_df.shape

(66849, 4)

In [26]:
X_train, y_train = preparing_train_df(train_df)

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
X_validat, y_validat  = preparing_validation_df(validation_df)

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
X_test = preparing_test_df(test_df)

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# X_train_price = train_df['price'].values
# X_validat_price = validation_df['price'].values

In [30]:
# scaler = StandardScaler()
# X_train_price = scaler.fit_transform(X_test_price.reshape(-1, 1))  
# X_test_price = scaler.transform(X_test_price.reshape(-1, 1))  

In [31]:
X_train.shape[0]


66849

In [32]:
# X_train_price.shape[0]

In [33]:
import tensorflow as tf
from tensorflow import keras

num_classes = len(set(y_train))  


# Define the model
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001), input_shape=(X_train.shape[1],)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),

    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(num_classes, activation='softmax')  
])


optimizer = keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.01)


model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)


model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_validat, y_validat), callbacks=[early_stopping, reduce_lr])


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.5105 - loss: 3.8629 - val_accuracy: 0.9535 - val_loss: 0.9790 - learning_rate: 5.0000e-04
Epoch 2/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.9669 - loss: 0.3457 - val_accuracy: 0.9782 - val_loss: 0.1623 - learning_rate: 5.0000e-04
Epoch 3/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.9822 - loss: 0.1725 - val_accuracy: 0.9818 - val_loss: 0.1399 - learning_rate: 5.0000e-04
Epoch 4/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.9880 - loss: 0.1246 - val_accuracy: 0.9829 - val_loss: 0.1305 - learning_rate: 5.0000e-04
Epoch 5/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.9906 - loss: 0.1059 - val_accuracy: 0.9841 - val_loss: 0.1220 - learning_rate: 5.0000e-04
Epoch 6/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.9901 - loss: 0.0957 - val_accuracy: 0.9856 - val_loss: 0.1146 - learning_rate: 5.0000e-04
Epoch 7/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/ste

In [34]:
train_loss, train_acc = model.evaluate(X_train, y_train)
print(f"Train Accuracy: {train_acc:.4f}")

validation_loss, validation_acc = model.evaluate(X_validat, y_validat)
print(f"Test Accuracy: {validation_acc:.4f}")

2090/2090 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9972 - loss: 0.0147
Train Accuracy: 0.9972
523/523 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9897 - loss: 0.0612
Test Accuracy: 0.9901


In [40]:
probabilities = model.predict(X_test)

confidence_scores = np.max(probabilities, axis=1)

predicted_indices = np.argmax(probabilities, axis=1)

predicted_labels = le.inverse_transform(predicted_indices)

confidence_threshold = 0.75

test_results = []

for i in range(len(X_test)):
    confidence = confidence_scores[i]
    predicted_class = predicted_labels[i]

    if confidence < confidence_threshold:
        predicted_class = "Unknown"

    test_results.append({
        'Predicted': predicted_class,
        'Confidence': f"{confidence:.2f}"
    })

temp_df = pd.DataFrame(test_results)

temp_df.head()


2090/2090 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


,Predicted,Confidence
0,1312,1.00
1,4743,1.00
2,2517,1.00
3,2374,1.00
4,639,1.00


In [41]:
predicted_df = test_df

In [42]:
len(predicted_df) 

66849

In [43]:
len(temp_df) 

66849

In [44]:
predicted_df['Predicted'] = temp_df['Predicted'].values
predicted_df['Confidence'] = temp_df['Confidence'].values
predicted_df.head()

,seller_item_name,price,Predicted,Confidence
0,موزابرايد 5 مجم ق 3 ش/ويسترن,91.5,1312,1.00
1,ميكروسرك 16 اقراص,29.0,4743,1.00
2,البافيت كالسيوم شراب,26.0,2517,1.00
3,زيستريل 20 مجم اقراص,52.0,2374,1.00
4,اتور 10مجم اقراص,33.0,639,1.00


In [57]:
predicted_df.to_csv('predicted.csv', index=False, encoding='utf-8-sig')

In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU

num_classes = len(set(y_train))  

# Define the model
model = keras.Sequential([
    Dense(512, kernel_regularizer=keras.regularizers.l2(0.0001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),  # LeakyReLU instead of ReLU
    Dropout(0.3),

    Dense(256, kernel_regularizer=keras.regularizers.l2(0.0001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.25),

    Dense(128, kernel_regularizer=keras.regularizers.l2(0.0001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),

    Dense(num_classes, activation='softmax')  
])

# Optimizer with weight decay
optimizer = keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.01)

# Compile model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Improved Early Stopping - Now monitors accuracy instead of loss
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, mode='max')

# Learning Rate Reduction
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1, mode='max')

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=50, batch_size=128, 
                    validation_data=(X_test, y_test), 
                    callbacks=[early_stopping, reduce_lr])


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


NameError: name 'y_test' is not defined

In [ ]:
train_loss, train_acc = model.evaluate(X_train, y_train)
print(f"Train Accuracy: {train_acc:.4f}")

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

2090/2090 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9971 - loss: 0.0433
Train Accuracy: 0.9969
523/523 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9896 - loss: 0.0869
Test Accuracy: 0.9887


In [37]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, BatchNormalization

num_classes = len(set(y_train))  

# Reshape data for LSTM compatibility
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the RNN model
model = keras.Sequential([
    Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.3)),  
    BatchNormalization(),
    Dropout(0.3),
    
    Bidirectional(LSTM(128, return_sequences=False, recurrent_dropout=0.2)),  
    BatchNormalization(),
    Dropout(0.2),

    Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
    BatchNormalization(),
    Dropout(0.2),

    Dense(num_classes, activation='softmax')  
])

# Optimizer
optimizer = keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.005)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train
history = model.fit(X_train_reshaped, y_train, 
                    epochs=50, batch_size=128, 
                    validation_data=(X_test_reshaped, y_test), 
                    callbacks=[early_stopping, reduce_lr])


Epoch 1/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 36s 61ms/step - accuracy: 0.5942 - loss: 3.2873 - val_accuracy: 0.9379 - val_loss: 2.2854 - learning_rate: 5.0000e-04
Epoch 2/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 32s 62ms/step - accuracy: 0.9617 - loss: 0.2525 - val_accuracy: 0.9745 - val_loss: 0.1182 - learning_rate: 5.0000e-04
Epoch 3/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 32s 61ms/step - accuracy: 0.9785 - loss: 0.1082 - val_accuracy: 0.9762 - val_loss: 0.0944 - learning_rate: 5.0000e-04
Epoch 4/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 32s 61ms/step - accuracy: 0.9837 - loss: 0.0721 - val_accuracy: 0.9782 - val_loss: 0.0851 - learning_rate: 5.0000e-04
Epoch 5/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 32s 61ms/step - accuracy: 0.9867 - loss: 0.0562 - val_accuracy: 0.9798 - val_loss: 0.0854 - learning_rate: 5.0000e-04
Epoch 6/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 32s 62ms/step - accuracy: 0.9885 - loss: 0.0482 - val_accuracy: 0.9798 - val_loss: 0.0782 - learning_rate: 5.0000e-04
Epoch 7/50
523/523 ━━━━━━━━━━━━━━━━━━━━ 35s 67ms/ste

In [38]:
train_loss, train_acc = model.evaluate(X_train_reshaped, y_train)
print(f"Train Accuracy: {train_acc:.4f}")

test_loss, test_acc = model.evaluate(X_test_reshaped, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

2090/2090 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9968 - loss: 0.0132
Train Accuracy: 0.9966
523/523 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9879 - loss: 0.0555
Test Accuracy: 0.9873
